In [ ]:
import networkx as nx
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt
from descartes import PolygonPatch
from shapely.geometry import Point, LineString, Polygon
from shapely.ops import cascaded_union
import urllib.request
import pandas as pd
import zipfile
import folium
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from branca.element import Template, MacroElement

import plotly
import plotly.graph_objects as go


In [ ]:
## https://github.com/nick-caros/nyc-travel-times
## https://towardsdatascience.com/finding-time-dependent-travel-times-between-every-pair-of-locations-in-manhattan-c3c48b0db7ba
##http://pysal.org/
##https://nathanrooy.github.io/posts/2021-03-12/osmnx-openstreetmap-taxicab-routing/
##https://stackoverflow.com/questions/62156969/osmnx-shortest-path-nodes-get-node-traveled-time

In [ ]:
point = 51.515472, -0.141336 #-------------- Oxford Circus Underground tubestop

G = ox.graph.graph_from_point(point, dist=3000, simplify=True,  network_type='drive')

# If a node cannot access at least 10% of other nodes, delete it (isolated) 
# remove_list = []
# num_nodes = len(G.nodes)
# for node in G.nodes:  
#     reach = len(nx.descendants(G, node))
#     if reach < num_nodes / 10:
#         remove_list.append(node)

# for node in remove_list:
#     G.remove_node(node)

# change node labels to integers
G = nx.convert_node_labels_to_integers(G, label_attribute='old_node_ID')
ox.plot_graph(G, figsize = (10,10));

In [ ]:
### Use freeflow travel time as backup in case travel time not available in Uber data
G = ox.add_edge_speeds(G)
G = ox.speed.add_edge_travel_times(G, precision=1)

In [ ]:
# Use Uber Movement data to get average speed for all ways during each hour of the average weekday
speed_df = pd.read_csv('data\movement-speeds-quarterly-by-hod-london-2020-Q1.csv') # downloaded from Uber Movement
speed_df_1 = speed_df.groupby(["osm_way_id", "hour_of_day"]).mean().reset_index()
speed_df_2 = speed_df_1[['osm_way_id', 'hour_of_day', 'speed_mph_mean']]
speed_df_2['speed'] = (speed_df_2['speed_mph_mean'] * 1.60934).round(1)
speed_df_3 = speed_df_2[(speed_df_2['hour_of_day'] >= 8) & (speed_df_2['hour_of_day'] <= 19)].reset_index(drop = True) #---- GP practice opening times 
speed_df_4 = speed_df_3.drop('speed_mph_mean', axis = 1)

In [ ]:
# Convert to dictionary with way IDs and hour of the day as keys
speed_dict = dict([((t.osm_way_id, t.hour_of_day), t.speed) for t in speed_df_4.itertuples()])

In [ ]:
#### 

hour_choice = 18 #-----------chosing 6 pm 
for edge in G.edges:
    edge_obj = G[edge[0]][edge[1]][edge[2]]
    wayid = edge_obj['osmid']
    try:
        speed = speed_dict[(wayid,hour_choice)]
        distance = edge_obj['length'] / 1000 # Convert from m to km
        travel_time = distance / speed * 60 # Convert from hours to minutes
    except:
        travel_time = edge_obj['travel_time'] / 60 # Convert from seconds to minutes
    G[edge[0]][edge[1]][edge[2]]['uber_travel_time'] = travel_time


In [ ]:
frame = folium.Figure(width=500, height=500)
route_map = folium.Map(
    location= point,
    tiles="cartodbpositron",
    zoom_start=12).add_to(frame)

In [ ]:

origin_location = ['St. John & St Elizabeth Hospital', '60 Grove End Rd, London, NW8 9NH', 51.5329, -0.1744]
destination_location = ['St. Thomas Hospital', 'Westminster Bridge Rd, London, SE1 7EH', 51.4991,  -0.11706]
graph_map = ox.plot_graph_folium(G, route_map, color="#425563", opacity=0.3, weight=1)
orig = ox.distance.nearest_nodes(G, Y = origin_location[2], X = origin_location[3])
dest = ox.distance.nearest_nodes(G, Y = destination_location[2], X =  destination_location[3])
route = nx.shortest_path(G, orig, dest, 'travel_time')
route_uber = nx.shortest_path(G, orig, dest, 'uber_travel_time')
route_map = ox.plot_route_folium(G, route, route_map=route_map, color='#41B6E6')
route_map = ox.plot_route_folium(G, route_uber, route_map=route_map, color='#003087')
folium.Marker(origin_location[2:], popup = folium.Popup('<b>' + 'Name: ' + '</b>'  + origin_location[0] + '<br>'
        '<b>' + 'Address: ' + '</b>'  + origin_location[1] + '<br>',
        max_width=len(origin_location[1])*20),
        icon=folium.Icon(color="darkblue", prefix='fa', icon="fa-h-square"),
        ).add_to(route_map)
folium.Marker(destination_location[2:],popup = folium.Popup('<b>' + 'Name: ' + '</b>'  + destination_location[0] + '<br>'
        '<b>' + 'Address: ' + '</b>'  + destination_location[1] + '<br>',
        max_width=len(destination_location[1])*20),
        icon=folium.Icon(color="blue", prefix='fa', icon="fa-h-square"),
        ).add_to(route_map)
route_map

In [ ]:
##CSS styling Plot 2 legend
template = """
{% macro html(this, kwargs) %}
<!doctype html>
<html lang="en">
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:5px; padding: 3px; font-size:14px; right: 8px; top: 8px;'>
     
<div class='legend-title'>Routes:</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#41B6E6'></span><p>&nbsp;&nbsp;</p>
Free-flow<p>&nbsp;&nbsp;</p></li>
    <li><span style='background:#003087;'></span><p>&nbsp;&nbsp;</p>
Uber weighted</li>
  </ul>
</body>
</html>
<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 85%;
    }
  .maplegend .legend-text {
    text-align: left;
    margin-bottom: 5px;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels {
    display: flex;
    justify-content: space-between;
    width: 100%;
  }
  .maplegend ul.legend-labels li {
    display: flex;
    align-items: center;
  }
  .maplegend ul.legend-labels li span {
    display: block;
    height: 15px;
    width: 25px;
  }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""
macro = MacroElement()
macro._template = Template(template)
route_map.get_root().add_child(macro)

In [ ]:

# - Free-flow route
cols = ['osmid', 'length', 'travel_time', 'uber_travel_time']
attrs_ff = ox.utils_graph.get_route_edge_attributes(G, route)
df_ff = pd.DataFrame(attrs_ff)[cols]

# - Uber-weighted route
attrs_uber = ox.utils_graph.get_route_edge_attributes(G, route_uber)
df_uber = pd.DataFrame(attrs_uber)[cols]


In [ ]:
## The speed limit for the (A4202) route is 30 mph

data = [['Free-flow', (df_ff['length'].sum()/1000), (df_ff['travel_time'].sum())/60, df_ff['uber_travel_time'].sum()], 
        ['Uber weighted', (df_uber['length'].sum()/1000), (df_uber['travel_time'].sum())/60, df_uber['uber_travel_time'].sum()]]
df_travel = pd.DataFrame(data, columns = ['Route', 'Distance (km)', 'Free-flow Travel time (min)', 'Uber weighted travel time (min)'])
df_travel = df_travel.round(2)
df_travel

In [ ]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_travel.columns),
                fill_color='#003087',
                line_color='Black',
                font=dict(color='white', size=12),
                align='center'),
    cells=dict(values= [df_travel['Route'], df_travel['Distance (km)'], df_travel['Free-flow Travel time (min)'], df_travel['Uber weighted travel time (min)']],
               fill_color='white',
               line_color='Black',
               font=dict(color='Black', size=12),
               align='center'))
])
fig.show()

In [ ]:

from plotly.subplots import make_subplots

In [ ]:

fig = make_subplots(rows=1, cols=3, shared_yaxes=False)
fig.add_trace(go.Bar(x=df_travel['Route'], y=df_travel['Distance (km)']),
              1, 1)
fig.add_trace(go.Bar(x=df_travel['Route'], y = df_travel['Free-flow Travel time (min)']),
              1, 2)
fig.add_trace(go.Bar(x=df_travel['Route'], y=df_travel['Uber weighted travel time (min)']),
              1, 3)
# Update xaxis properties
fig.update_xaxes(title_text="Route", row=1, col=1)
fig.update_xaxes(title_text="Route", row=1, col=2)
fig.update_xaxes(title_text="Route", row=1, col=3)


# Update yaxis properties
fig.update_yaxes(title_text="Distance (km)", row=1, col=1)
fig.update_yaxes(title_text="Free-flow Travel time (min)", row=1, col=2)
fig.update_yaxes(title_text="Uber weighted travel time (min)", row=1, col=3)

fig.show()